### **문제 1) Tokenizer 생성하기**

**1-1. `preprocessing()`**

텍스트 전처리를 하는 함수입니다.

- input: 여러 영어 문장이 포함된 list 입니다. ex) ['I go to school.', 'I LIKE pizza!']
- output: 각 문장을 토큰화한 결과로, nested list 형태입니다. ex) [['i', 'go', 'to', 'school'], ['i', 'like', 'pizza']]
- 조건 1: 입력된 문장에 대해서 소문자로의 변환과 특수문자 제거를 수행합니다.
- 조건 2: 토큰화는 white space 단위로 수행합니다.
    
    

**1-2. `fit()`**

어휘 사전을 구축하는 함수입니다.

- input: 여러 영어 문장이 포함된 list 입니다. ex) ['I go to school.', 'I LIKE pizza!']
- 조건 1: 위에서 만든 `preprocessing` 함수를 이용하여 각 문장에 대해 토큰화를 수행합니다.
- 조건 2: 각각의 토큰을 정수 인덱싱 하기 위한 어휘 사전(`self.word_dict`)을 생성합니다.
    - 주어진 코드에 있는 `self.word_dict`를 활용합니다.
    

**1-3. `transform()`**

어휘 사전을 활용하여 입력 문장을 정수 인덱싱하는 함수입니다.

- input: 여러 영어 문장이 포함된 list입니다. ex) ['I go to school.', 'I LIKE pizza!']
- output: 각 문장의 정수 인덱싱으로, nested list 형태입니다. ex) [[1, 2, 3, 4], [1, 5, 6]]
- 조건 1: 어휘 사전(`self.word_dict`)에 없는 단어는 'oov'의 index로 변환합니다.

In [107]:
class Tokenizer():
  def __init__(self):
    self.word_dict = {'oov': 0}
    self.fit_checker = False
  
  def preprocessing(self, sequences):
    result = []
    '''
    문제 1-1.
    '''
    import re
    for sentence in sequences:
      result.append(re.sub(r"[^a-z&\s]","",sentence.lower()).split())
    return result
  
  def fit(self, sequences):
    self.fit_checker = False
    '''
    문제 1-2.
    '''
    self.word_dict = {'oov': 0}
    num=1
    for i in self.preprocessing(sequences):
      for j in i:
        if j not in self.word_dict:
          self.word_dict[j]=num
          num+=1
    self.fit_checker = True
  
  def transform(self, sequences):
    result = []
    tokens = self.preprocessing(sequences)
    if self.fit_checker:
      '''
      문제 1-3.
      '''
      for sentence in tokens:
          result.append(list(map(lambda x: self.word_dict[x] if x in self.word_dict else self.word_dict['oov'],sentence)))
      return result
    else:
      raise Exception("Tokenizer instance is not fitted yet.")
      
  def fit_transform(self, sequences):
    self.fit(sequences)
    result = self.transform(sequences)
    return result

### **문제 2) TfidfVectorizer 생성하기**

**2-1. `fit()`**

입력 문장들을 이용해 IDF 행렬을 만드는 함수입니다.

- input: 여러 영어 문장이 포함된 list 입니다. ex) ['I go to school.', 'I LIKE pizza!']
- 조건 1: IDF 행렬은 list 형태입니다.
    - ex) [토큰1에 대한 IDF 값, 토큰2에 대한 IDF 값, .... ]
- 조건 2: IDF 값은 아래 식을 이용해 구합니다.
    
    $$
    idf(d,t)=log_e(\frac{n}{1+df(d,t)})
    $$
    
    - $df(d,t)$ : 단어 t가 포함된 문장 d의 개수
    - $n$ : 입력된 전체 문장 개수
- 조건 3: 입력된 문장의 토큰화에는 문제 1에서 만든 Tokenizer를 사용합니다.
    
    

**2-2. `transform()`**

입력 문장들을 이용해 TF-IDF 행렬을 만드는 함수입니다.

- input: 여러 영어 문장이 포함된 list입니다. ex) ['I go to school.', 'I LIKE pizza!']
- output : nested list 형태입니다.
    
    ex) [[tf-idf(1, 1), tf-idf(1, 2), tf-idf(1, 3)], [tf-idf(2, 1), tf-idf(2, 2), tf-idf(2, 3)]]
    
    |  | 토큰1 | 토큰2 | 토큰3 |
    | --- | --- | --- | --- |
    | 문장1 | tf-idf(1,1) | tf-idf(1,2) | tf-idf(1,3) |
    | 문장2 | tf-idf(2,1) | tf-idf(2,2) | tf-idf(2,3) |
- 조건1 : 입력 문장을 이용해 TF 행렬을 만드세요.
    - $tf(d, t)$ : 문장 d에 단어 t가 나타난 횟수
- 조건2 : 문제 2-1( `fit()`)에서 만든 IDF 행렬과 아래 식을 이용해 TF-IDF 행렬을 만드세요
    
    $$
    tf-idf(d,t) = tf(d,t) \times idf(d,t)
    $$

In [108]:
class TfidfVectorizer:
  def __init__(self, tokenizer):
    self.tokenizer = tokenizer()
    self.fit_checker = False
  
  def fit(self, sequences):
    tokenized = self.tokenizer.fit_transform(sequences)
    '''
    문제 2-1.
    '''
    import numpy as np
    self.idf=list(self.tokenizer.word_dict.values())

    cnt = [[0 for x in self.idf] for x in tokenized]
    for n,i in enumerate(tokenized):
      for j in i:
        cnt[n][j]+=1

    n_doc=len(tokenized)
    for i in range(len(self.idf)):
      count=0
      for j in cnt:
        if j[self.idf[i]]:
          count +=1
      self.idf[i]=np.log(n_doc/(1+count))
    self.fit_checker = True
    

  def transform(self, sequences):
    if self.fit_checker:
      tokenized = self.tokenizer.transform(sequences)
      '''
      문제 2-2.
      '''
      self.tfidf_matrix = [[0 for x in self.idf] for x in tokenized]
      for n,i in enumerate(tokenized):
        for j in i:
          self.tfidf_matrix[n][j]+=1
      for i in range(len(self.tfidf_matrix)):
        for j in range(len(self.tfidf_matrix[i])):
          self.tfidf_matrix[i][j]=self.tfidf_matrix[i][j]*self.idf[j]
      return self.tfidf_matrix
    else:
      raise Exception("TfidfVectorizer instance is not fitted yet.")

  
  def fit_transform(self, sequences):
    self.fit(sequences)
    return self.transform(sequences)

In [111]:
example='In the 1600s the Dutch East India Company employed hundreds of ships to trade gold, porcelain, spices, and silks around the globe. But running this massive operation wasn’t cheap. In order to fund their expensive voyages, the company turned to private citizens– individuals who could invest money to support the trip in exchange for a share of the ship’s profits. This practice allowed the company to afford even grander voyages, increasing profits for both themselves and their savvy investors.'.split('.')
print(example)

['In the 1600s the Dutch East India Company employed hundreds of ships to trade gold, porcelain, spices, and silks around the globe', ' But running this massive operation wasn’t cheap', ' In order to fund their expensive voyages, the company turned to private citizens– individuals who could invest money to support the trip in exchange for a share of the ship’s profits', ' This practice allowed the company to afford even grander voyages, increasing profits for both themselves and their savvy investors', '']


In [110]:
tfidf=TfidfVectorizer(Tokenizer)
tfidf.fit(example)
tfidf.transform(example)

[[0.0,
  0.5108256237659907,
  0.6694306539426294,
  0.9162907318741551,
  0.9162907318741551,
  0.9162907318741551,
  0.9162907318741551,
  0.22314355131420976,
  0.9162907318741551,
  0.9162907318741551,
  0.5108256237659907,
  0.5108256237659907,
  0.22314355131420976,
  0.9162907318741551,
  0.9162907318741551,
  0.9162907318741551,
  0.9162907318741551,
  0.5108256237659907,
  0.9162907318741551,
  0.9162907318741551,
  0.9162907318741551,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0],
 [0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.9162907318741551,
  0.9162907318741551,
  0.5108256237659907,
  0.9162907318741551,
  0.9162907318741551,
  0.9162907318741551,
  0.91629073